SIRS'I' - Basic Model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#Define the differential equations
def dSdt(t,S,I,R,s,i):
    return -beta*S*(I+i)/N + sigma*N -sigma*S

def dIdt(t,S,I,R,s,i):
    return beta*S*(I+i)/N - gamma*I- sigma*I

def dRdt(t,S,I,R,s,i):
    return gamma*(I+i)-epsilon*R-sigma*R

def dsdt(t,S,I,R,s,i):
    return -beta*s*(I+i)/N + epsilon*R -sigma*s

def didt(t,S,I,R,s,i):
    return beta*s*(I+i)/N - gamma*i -sigma*i

#Define parameters
t0,tf = 0,360*30
dt = 1
nsteps = int((tf-t0)/dt)
beta0 = 0.4
delta = 0.1
gamma = 0.07
epsilon = 1/(7*360)
N = 7000000
sigma = 200/N
I0 = 307818
R0 = 0
s0 = 0
i0 = 31669
S0 = N-I0-R0-s0-i0

#Define arrays to store solutions
t = np.linspace(t0,tf,nsteps+1)
S = np.zeros(nsteps+1)
I = np.zeros(nsteps+1)
R = np.zeros(nsteps+1)
s = np.zeros(nsteps+1)
i = np.zeros(nsteps+1)

S[0], I[0], R[0], s[0], i[0] = S0,I0,R0,s0,i0


for n in range(nsteps):
    beta = beta0+delta*np.cos(2*np.pi*t[n]/360 + np.pi) #+ np.pi)
    S[n+1] = S[n] + dSdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
    I[n+1] = I[n] + dIdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
    R[n+1] = R[n] + dRdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
    s[n+1] = s[n] + dsdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
    i[n+1] = i[n] + didt(t[n],S[n],I[n],R[n],s[n],i[n])*dt


# Plot I and I'
plt.figure(figsize=(10, 6))
plt.plot(t/360, I, label='Infected (I)')
plt.title('Dynamics of I (infected population) Over 30 Years')
plt.xlabel('Time (years)')
plt.ylabel('Number of Individuals')
plt.legend()
plt.grid(True)
plt.show()


plt.figure(figsize=(10, 6))
plt.plot(t/360, i, label='Re-infected (i)')
plt.title('Dynamics of the i (re-infected population) Over 30 years')
plt.xlabel('Time (years)')
plt.ylabel('Number of Individuals')
plt.legend()
plt.grid(True)
plt.show()

Optimization Code1:-Beta0 and Delta Variation

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap

df = pd.read_csv("E:\\BIOPHAM Master File\\Internship-Prof.Sergio\\Past 30years data\\data.csv")
infected = df.iloc[:, 0].tolist()
reinfected = df.iloc[:, 1].tolist()

#Define the differential equations
def dSdt(t,S,I,R,s,i):
    return -beta*S*(I+i)/N + sigma*N -sigma*S

def dIdt(t,S,I,R,s,i):
    return beta*S*(I+i)/N - gamma*I- sigma*I

def dRdt(t,S,I,R,s,i):
    return gamma*(I+i)-epsilon*R-sigma*R

def dsdt(t,S,I,R,s,i):
    return -0.8*beta*s*(I+i)/N + epsilon*R -sigma*s

def didt(t,S,I,R,s,i):
    return beta*s*(I+i)/N - gamma*i -sigma*i

#Define parameters
t0,tf = 0,360*30
dt = 1
nsteps = int((tf-t0)/dt)
N = 7618000
sigma = 200/N
I0 = 1000
R0 = 0
s0 = 0
i0 = 100
S0 = N-I0-R0-s0-i0

#Arrays of constants - beta0, slope, epsilon,gamma
Beta0 = [0]*61
Delta = [0]*61
gamma = 0.07
epsilon = 1/(7*360)
Beta0[30] = 0.4
Delta[30] = 0.1

m = 1
for c in range(30,61):
    Beta0[c] = Beta0[30]*m
    Delta[c] = Delta[30]*m
    m+=0.04

m = 1
for c in range(29,-1,-1):
    Beta0[c] = Beta0[30]*m
    Delta[c] = Delta[30]*m
    m-=0.04

Beta0_grid, Delta_grid = np.meshgrid(Beta0, Delta)

#Define arrays to store solutions
t = np.linspace(t0,tf,nsteps+1)
S = np.zeros(nsteps+1)
I = np.zeros(nsteps+1)
R = np.zeros(nsteps+1)
s = np.zeros(nsteps+1)
i = np.zeros(nsteps+1)
S[0], I[0], R[0], s[0], i[0] = S0,I0,R0,s0,i0

error_grid = np.zeros_like(Beta0_grid)


for a in range(Beta0_grid.shape[0]):
    for b in range(Beta0_grid.shape[1]):
        beta0 = Beta0_grid[a,b]
        delta = Delta_grid[a,b]
        for n in range(nsteps):
            beta = beta0+delta*np.cos(2*np.pi*t[n]/360 + np.pi)
            S[n+1] = S[n] + dSdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
            I[n+1] = I[n] + dIdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
            R[n+1] = R[n] + dRdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
            s[n+1] = s[n] + dsdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
            i[n+1] = i[n] + didt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
        infectedError = np.sum(np.abs(I[29*360:]-infected[29*360:]))
        reinfectedError = np.sum(np.abs(I[29*360:]-reinfected[29*360:]))
        error_grid[a,b] = infectedError + reinfectedError
min = np.min(error_grid)
index=np.unravel_index(np.argmin(error_grid), error_grid.shape)
print(f'min={min}, index={index}')

#Create color map
cmap = LinearSegmentedColormap.from_list('GreenRed',['green','yellow','red'])

plt.figure(figsize=(10,8))
plt.pcolor(Delta_grid, Beta0_grid, error_grid, shading='auto',cmap=cmap)
plt.colorbar(label='Absolute Error')
plt.xlabel('Delta')
plt.ylabel('Beta0')
plt.title('Absolute error calculated between I & I\' of calculated and complex model', pad=2)
plt.tight_layout()
plt.show()

Optimization Code2:-Epsilon and Gamma variation

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap

df = pd.read_csv("E:\\BIOPHAM Master File\\Internship-Prof.Sergio\\Past 30years data\\data.csv")
infected = df.iloc[:, 0].tolist()
reinfected = df.iloc[:, 1].tolist()

#Define the differential equations
def dSdt(t,S,I,R,s,i):
    return -beta*S*(I+i)/N + sigma*N -sigma*S

def dIdt(t,S,I,R,s,i):
    return beta*S*(I+i)/N - gamma*I- sigma*I

def dRdt(t,S,I,R,s,i):
    return gamma*(I+i)-epsilon*R-sigma*R

def dsdt(t,S,I,R,s,i):
    return -beta*s*(I+i)/N + epsilon*R -sigma*s

def didt(t,S,I,R,s,i):
    return beta*s*(I+i)/N - gamma*i -sigma*i

#Define parameters
t0,tf = 0,360*30
dt = 1
nsteps = int((tf-t0)/dt)
N = 7000000
sigma = 200/N
I0 = 1000
R0 = 0
s0 = 0
i0 = 100
S0 = N-I0-R0-s0-i0

#Arrays of constants - beta0, slope, epsilon,gamma
Gamma = [0]*61
Epsilon = [0]*61
Gamma[30] = 0.08208
Epsilon[30] = 0.00036783
beta0 = 0.9
delta = 0.24

m = 1
for c in range(30,61):
    Gamma[c] = Gamma[30]*m
    Epsilon[c] = Epsilon[30]*m
    m+=0.005

m = 1
for c in range(29,-1,-1):
    Gamma[c] = Gamma[30]*m
    Epsilon[c] = Epsilon[30]*m
    m-=0.005

Gamma_grid, Epsilon_grid = np.meshgrid(Gamma, Epsilon)

#Define arrays to store solutions
t = np.linspace(t0,tf,nsteps+1)
S = np.zeros(nsteps+1)
I = np.zeros(nsteps+1)
R = np.zeros(nsteps+1)
s = np.zeros(nsteps+1)
i = np.zeros(nsteps+1)
S[0], I[0], R[0], s[0], i[0] = S0,I0,R0,s0,i0

error_grid = np.zeros_like(Gamma_grid)


for a in range(Gamma_grid.shape[0]):
    for b in range(Gamma_grid.shape[1]):
        gamma = Gamma_grid[a,b]
        epsilon = Epsilon_grid[a,b]
        for n in range(nsteps):
            beta = beta0+delta*np.cos(2*np.pi*t[n]/360 + np.pi)
            S[n+1] = S[n] + dSdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
            I[n+1] = I[n] + dIdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
            R[n+1] = R[n] + dRdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
            s[n+1] = s[n] + dsdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
            i[n+1] = i[n] + didt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
        infectedError = np.sum(np.abs(I[29*360:]-infected[29*360:]))
        reinfectedError = np.sum(np.abs(I[29*360:]-reinfected[29*360:]))
        error_grid[a,b] = infectedError + reinfectedError

min = np.min(error_grid)
index=np.unravel_index(np.argmin(error_grid), error_grid.shape)
print(f'min={min}, index={index}')
#Create color map
cmap = LinearSegmentedColormap.from_list('GreenRed',['green','yellow','red'])

plt.figure(figsize=(10,8))
plt.pcolor(Gamma_grid, Epsilon_grid, error_grid, shading='auto',cmap=cmap)
plt.colorbar(label='Absolute Error')
plt.xlabel('Gamma')
plt.ylabel('Epsilon')
plt.title('Absolute error calculated between I & I\' of calculated and complex model', pad=2)
plt.tight_layout()
plt.show()

COVID Model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv("E:\\BIOPHAM Master File\\Internship-Prof.Sergio\\Past 30years data\\data.csv")
infected = df.iloc[:, 0].tolist()
reinfected = df.iloc[:, 1].tolist()



#Define the differential equations
def dSdt(t,S,I,R,s,i):
    return -beta*S*(I+i)/N + sigma*N -sigma*S

def dIdt(t,S,I,R,s,i):
    return beta*S*(I+i)/N - gamma*I- sigma*I

def dRdt(t,S,I,R,s,i):
    return gamma*(I+i)-epsilon*R-sigma*R

def dsdt(t,S,I,R,s,i):
    return -beta*s*(I+i)/N + epsilon*R -sigma*s

def didt(t,S,I,R,s,i):
    return beta*s*(I+i)/N - gamma*i -sigma*i

#Define parameters
t0,tf = 0,360*30
dt = 1
nsteps = int((tf-t0)/dt)
gamma =   0.0701784
epsilon = 0.000332886
N = 7000000
sigma = 200/N
I0 = 307818
R0 = 0
s0 = 0
i0 = 31669
S0 = N-I0-R0-s0-i0
s0,sf =0.04,0.30
ds=0.02
steps = int((sf-s0)/ds)


#Define arrays to store solutions
t = np.linspace(t0,tf,nsteps+1)
S = np.zeros(nsteps+1)
I = np.zeros(nsteps+1)
R = np.zeros(nsteps+1)
s = np.zeros(nsteps+1)
i = np.zeros(nsteps+1)
p = np.zeros(nsteps+1)

S[0], I[0], R[0], s[0], i[0], p[0] = S0,I0,R0,s0,i0, N
error = [0]

#0.9,0.3,0.096, 0.000402 Param- 1
#0.9, 0.24, 0.08208, 0.00036783 Params-2
# 0.792,0.2016,0.0701784,0.000332886 - Params-3
beta0 = 0.792
slope=  0.2016
for n in range(nsteps):
    if n >= 9000 and n < 9200:
        beta = 0.7 * (beta0 + slope * np.cos(2 * np.pi * t[n] / 360 + np.pi))
    else:
        beta = beta0 + slope * np.cos(2 * np.pi * t[n] / 360 + np.pi)
    S[n+1] = S[n] + dSdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
    I[n+1] = I[n] + dIdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
    R[n+1] = R[n] + dRdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
    s[n+1] = s[n] + dsdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
    i[n+1] = i[n] + didt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
    p[n+1] = S[n+1] + I[n+1] + R[n+1] + s[n+1] + i[n+1]



start_index = 20 * 360
end_index = 30 * 360

# Slice the arrays to get the desired range
t_slice = t[start_index:end_index]
I_slice = I[start_index:end_index]
i_slice = i[start_index:end_index]
infected_slice = infected[start_index:end_index]
reinfected_slice = reinfected[start_index:end_index]


# Plotting the data
plt.plot(t_slice/360, I_slice, label='calculated I')
plt.plot(t_slice/360, infected_slice, label='complex model I')
plt.legend()
plt.xlabel('Time (scaled)')
plt.ylabel('Infection Levels')
plt.title(f'Infected Over Time, {beta0},{slope},{gamma},{epsilon}')
plt.show()

plt.plot(t_slice/360, i_slice, label='calculated i')
plt.plot(t_slice/360, reinfected_slice, label='complex model i')
plt.legend()
plt.xlabel('Time (scaled)')
plt.ylabel('Re-Infection Levels')
plt.title(f'Re-Infected Over Time, {beta0},{slope},{gamma},{epsilon}')
plt.show()

Vaccination model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv("E:\\BIOPHAM Master File\\Internship-Prof.Sergio\\Past 30years data\\data.csv")
infected = df.iloc[:, 0].tolist()
reinfected = df.iloc[:, 1].tolist()



#Define the differential equations
def dSdt(t,S,I,R,s,i):
    if n<26*360:
        return -beta*S*(I+i)/N + sigma*N -sigma*S
    else:
        return -beta*S*(I+i)/N + sigma*N*(1-neta) -sigma*S

def dIdt(t,S,I,R,s,i):
    return beta*S*(I+i)/N - gamma*I- sigma*I

def dRdt(t,S,I,R,s,i):
    if n<26*360:
        return gamma*(I+i)-epsilon*R-sigma*R
    else:
        return gamma*(I+i)-epsilon*R-sigma*R + sigma*N*neta

def dsdt(t,S,I,R,s,i):
    return -beta*s*(I+i)/N + epsilon*R -sigma*s

def didt(t,S,I,R,s,i):
    return beta*s*(I+i)/N - gamma*i -sigma*i

#Define parameters
t0,tf = 0,360*30
dt = 1
nsteps = int((tf-t0)/dt)
gamma =   0.0701784
epsilon = 0.000332886
N = 7000000
sigma = 200/N
I0 = 307818
R0 = 0
s0 = 0
i0 = 31669
S0 = N-I0-R0-s0-i0
s0,sf =0.04,0.30
ds=0.02
steps = int((sf-s0)/ds)


#Define arrays to store solutions
t = np.linspace(t0,tf,nsteps+1)
S = np.zeros(nsteps+1)
I = np.zeros(nsteps+1)
R = np.zeros(nsteps+1)
s = np.zeros(nsteps+1)
i = np.zeros(nsteps+1)
p = np.zeros(nsteps+1)

S[0], I[0], R[0], s[0], i[0], p[0] = S0,I0,R0,s0,i0, N
error = [0]


beta0 = 0.792
slope=  0.2016
neta=0.9
for n in range(nsteps):
    beta =beta0+slope*np.cos(2*np.pi*t[n]/360 + np.pi)
    S[n+1] = S[n] + dSdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
    I[n+1] = I[n] + dIdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
    R[n+1] = R[n] + dRdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
    s[n+1] = s[n] + dsdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
    i[n+1] = i[n] + didt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
    p[n+1] = S[n+1] + I[n+1] + R[n+1] + s[n+1] + i[n+1]



infectedError = np.sum(np.abs(I[29*360:]-infected[29*360:]))
reinfectedError = np.sum(np.abs(I[29*360:]-reinfected[29*360:]))
error = infectedError + reinfectedError


start_index = 20 * 360
end_index = 30 * 360

# Slice the arrays to get the desired range
t_slice = t[start_index:end_index]
I_slice = I[start_index:end_index]
i_slice = i[start_index:end_index]
infected_slice = infected[start_index:end_index]
reinfected_slice = reinfected[start_index:end_index]

# Plotting the data
plt.plot(t_slice/360, I_slice, label='calculated I')
plt.plot(t_slice/360, infected_slice, label='complex model I')
plt.legend()
plt.xlabel('Time (scaled)')
plt.ylabel('Infection Levels')
plt.title(f'Infected Over Time, {beta0},{slope},{gamma},{epsilon}')
plt.show()

plt.plot(t_slice/360, i_slice, label='calculated i')
plt.plot(t_slice/360, reinfected_slice, label='complex model i')
plt.legend()
plt.xlabel('Time (scaled)')
plt.ylabel('Re-Infection Levels')
plt.title(f'Re-Infected Over Time, {beta0},{slope},{gamma},{epsilon}')
plt.show()

Beta and Beta' for I and I'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv("E:\\BIOPHAM Master File\\Internship-Prof.Sergio\\Past 30years data\\data.csv")
infected = df.iloc[:, 0].tolist()
reinfected = df.iloc[:, 1].tolist()



#Define the differential equations
def dSdt(t,S,I,R,s,i):
    return -beta*S*(I+i)/N + sigma*N -sigma*S

def dIdt(t,S,I,R,s,i):
    return beta*S*(I+i)/N - gamma*I- sigma*I

def dRdt(t,S,I,R,s,i):
    return gamma*(I+i)-epsilon*R-sigma*R

def dsdt(t,S,I,R,s,i):
    return -neta*beta*s*(I+i)/N + epsilon*R -sigma*s

def didt(t,S,I,R,s,i):
    return neta*beta*s*(I+i)/N - gamma*i -sigma*i

#Define parameters
t0,tf = 0,360*30
dt = 1
nsteps = int((tf-t0)/dt)
gamma =   0.0701784
epsilon = 0.000332866
N = 7000000
sigma = 200/N
I0 = 307818
R0 = 0
s0 = 0
i0 = 31669
S0 = N-I0-R0-s0-i0
s0,sf =0.04,0.30
ds=0.02
steps = int((sf-s0)/ds)


#Define arrays to store solutions
t = np.linspace(t0,tf,nsteps+1)
S = np.zeros(nsteps+1)
I = np.zeros(nsteps+1)
R = np.zeros(nsteps+1)
s = np.zeros(nsteps+1)
i = np.zeros(nsteps+1)
p = np.zeros(nsteps+1)

S[0], I[0], R[0], s[0], i[0], p[0] = S0,I0,R0,s0,i0, N
error = [0]

#0.9,0.3,0.096, 0.000402 Param- 1
#0.9, 0.24, 0.08208, 0.00036783 Params-2
# 0.792,0.2016,0.0701784,0.000332886 - Params-3
beta0 = 0.792
slope=  0.2016
neta = 0.97
for _ in range(12):
    print(neta)
    for n in range(nsteps):
        beta =beta0+slope*np.cos(2*np.pi*t[n]/360 + np.pi)
        S[n+1] = S[n] + dSdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
        I[n+1] = I[n] + dIdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
        R[n+1] = R[n] + dRdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
        s[n+1] = s[n] + dsdt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
        i[n+1] = i[n] + didt(t[n],S[n],I[n],R[n],s[n],i[n])*dt
        p[n+1] = S[n+1] + I[n+1] + R[n+1] + s[n+1] + i[n+1]



    infectedError = np.sum(np.abs(I[29*360:]-infected[29*360:]))
    reinfectedError = np.sum(np.abs(I[29*360:]-reinfected[29*360:]))
    error = infectedError + reinfectedError
    neta+=0.03

    start_index = 20 * 360
    end_index = 30 * 360

    # Slice the arrays to get the desired range
    t_slice = t[start_index:end_index]
    I_slice = I[start_index:end_index]
    i_slice = i[start_index:end_index]
    infected_slice = infected[start_index:end_index]
    reinfected_slice = reinfected[start_index:end_index]


    print(error)

    # Plotting the data
    plt.plot(t_slice/360, infected_slice, label='complex model I')
    plt.plot(t_slice/360, I_slice, label='calculated I')
    plt.legend()
    plt.xlabel('Time (scaled)')
    plt.ylabel('Infection Levels')
    plt.title(f'Infected Over Time, {beta0},{slope},{gamma},{epsilon},n={neta}')
    plt.show()


    plt.plot(t_slice/360, reinfected_slice, label='complex model i')
    plt.plot(t_slice/360, i_slice, label='calculated i')
    plt.legend()
    plt.xlabel('Time (scaled)')
    plt.ylabel('Re-Infection Levels')
    plt.title(f'Re-Infected Over Time, {beta0},{slope},{gamma},{epsilon},n={neta}')
    plt.show()